In [1]:
import pandas as pd
import numpy as np
import re
import json
import matplotlib.pyplot as plt
import glob, os, csv

from langdetect import detect

import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.models import CoherenceModel
import ast
from pprint import pprint

import nltk
# nltk.download('stopwords')
# nltk.download('wordnet')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'http', 'javascript'])

from bs4 import BeautifulSoup

import pyLDAvis
import pyLDAvis.gensim

# import stanza
# # stanza.download('en')
# nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma')
import gc

In [2]:
def get_domain(url):
    url = str(url).replace('https://', '').replace('http://', '')
    result = re.findall('^(?:http?\:\/\/)?[\w\-\.]+\.onion', url)
    if len(result[0].split('.')) > 2:
        result[0] = ('.').join(result[0].split('.')[1:])
    if len(result) < 1:
        return 'NA'
    else:
        return result[0]

<>:3: DeprecationWarning: invalid escape sequence \:
<>:3: DeprecationWarning: invalid escape sequence \:
<>:3: DeprecationWarning: invalid escape sequence \:
<ipython-input-2-d3df041d8b58>:3: DeprecationWarning: invalid escape sequence \:
  result = re.findall('^(?:http?\:\/\/)?[\w\-\.]+\.onion', url)


In [3]:
# Convert html file to title and body texts
def body2text(file):
    html = open(file, "rb")
    soup = BeautifulSoup(html, "html.parser")
    body = []
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    
    for line in lines:
        if line:
            body.append(line)
    title = soup.title
    if title is None:
        title_text = None
    else:
        title_text = title.string
    html.close()
    
    return [title_text, body]

In [4]:
# convert html to string
def body2text_body(html):
    soup = BeautifulSoup(html)
    result = []
    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()

    text = soup.get_text()
    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    for line in lines:
        if line:
            result.append(line)

    return result

In [5]:
stemmer = PorterStemmer()
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text))

def preprocess(sentences):
    result = []
    
    for sent in sentences:
        lemmas = []
        # tokenize
        tokens = gensim.utils.simple_preprocess(sent)
        for token in tokens:
            lemma = lemmatize_stemming(token)
            # remove stopwords
            if lemma not in stop_words:
                lemmas.append(lemma)
        # POS tagging
        nltk_tagged = nltk.pos_tag(lemmas)
        for word, tag in nltk_tagged:
            # only keep nouns
            if tag.startswith('N'):
                result.append(word)
    return result

In [6]:
def language_detect(ls):
    text = ' '.join(ls)
    result = detect(text)
    return result

In [7]:
def get_domain_name(x, y):
    return x[y]

def custom_strip(x):
    return str(x).strip().strip('\n\t')

In [8]:
def most_frequent(List): 
    counter = 0
    num = List[0] 
      
    for i in List: 
        curr_frequency = List.count(i) 
        if(curr_frequency> counter): 
            counter = curr_frequency 
            num = i 
  
    return num 

In [9]:
date = 'mar-26-0'
df = pd.read_csv('../data/universe-'+date+'.csv')
df['body_text'] = df['body_text'].apply(ast.literal_eval)
df = df[df['status'] == 'live']
df = df.drop_duplicates(subset ='url', keep = 'first')
df['url'] = df['url'].apply(lambda x: x.replace('https://', '').replace('http://', ''))
df['domain_url'] = df['url'].apply(get_domain)
df.drop(columns=['status'], inplace=True)
df = df[['domain_url', 'url', 'title', 'body_text']]
df

,domain_url,url,title,body_text
0,bfnews3u2ox4m4ty.onion,bfnews3u2ox4m4ty.onion/article/ellencushing/bu...,14 Things BuzzFeed Tech Tried And Liked In 2016,[14 Things BuzzFeed Tech Tried And Liked In 20...
4,bfnews3u2ox4m4ty.onion,bfnews3u2ox4m4ty.onion/article/nicolenguyen/ip...,"I Used The New iPhone 7 And, TBH, Having No He...","[I Used The New iPhone 7 And, TBH, Having No H..."
12,bfnews3u2ox4m4ty.onion,bfnews3u2ox4m4ty.onion/amphtml/daveyalba/to-st...,"To Stay Relevant, Wireless Speaker Company Son...","[To Stay Relevant, Wireless Speaker Company So..."
15,bfnews3u2ox4m4ty.onion,bfnews3u2ox4m4ty.onion/article/nicolenguyen/ho...,Apple Just Released A New iPhone Battery Healt...,[Apple Just Released A New iPhone Battery Heal...
16,bfnews3u2ox4m4ty.onion,bfnews3u2ox4m4ty.onion/amphtml/nicolenguyen/ip...,"The iPhone XR Is Good, Which Means Choosing An...","[The iPhone XR Is Good, Which Means Choosing A..."
...,...,...,...,...
499995,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,<![CDATA[Galaxy3: Wire post by DarkLordZim]]>,[http://galaxy3bhpzxecbywoa2j4tg43muepnhfalars...
499996,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,<![CDATA[Galaxy3: Wire post by DarkLordZim]]>,[http://galaxy3bhpzxecbywoa2j4tg43muepnhfalars...
499997,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,<![CDATA[Galaxy3: Wire post by DarkLordZim]]>,[http://galaxy3bhpzxecbywoa2j4tg43muepnhfalars...
499998,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,galaxy3bhpzxecbywoa2j4tg43muepnhfalars4cce3fcx...,pianist's wire posts : Galaxy3,"[pianist's wire posts : Galaxy3, Galaxy3, Log ..."


In [10]:
split_num = 10
size = len(df)
df_list = []

for i in range(split_num):
    temp_df = df.iloc[int(i*size/split_num):int((i+1)*size/split_num), :]
    df_list.append(temp_df)

del df
gc.collect()

1500

In [11]:
processed_df_list = []
for df in df_list:
    # group urls, titles and body texts by domain
    df = df.groupby('domain_url').agg(list)
#     df.drop(columns=['url'], inplace=True)
    df.reset_index(inplace=True)

    df['body_text'] = df['body_text'].apply(lambda x: [z for y in x for z in y])
    # Only keep unique processed texts
    df['body_text'] = df['body_text'].apply(lambda x: list(set(x)))
    
#     # preprocess
#     df['body_token'] = df['body_text'].map(preprocess)
#     df = df[df['body_token'].str.len() != 0]
#     # detect language
#     df['language'] = df['body_token'].map(language_detect)
#     df.reset_index(inplace=True)
#     df.drop(columns='index' ,inplace=True)
    processed_df_list.append(df)
    df = None
    gc.collect()

In [12]:
df_all = pd.concat(processed_df_list)
del processed_df_list
gc.collect()
df_all

,domain_url,url,title,body_text
0,5figq755l7c55eopjphypkpfj5b4ap5nm6rvie2tygcwta...,[5figq755l7c55eopjphypkpfj5b4ap5nm6rvie2tygcwt...,[dillon | PedoBum],"[dillon | PedoBum, Home, Thanks for the visit...."
1,74i677egkh3zlk6r.onion,[74i677egkh3zlk6r.onion/pmwiki.php?n=PmWikiDe....,"[freiheitsfoo Wiki | PmWikiDe / Installation, ...","[# Erster Punkt, Powerweb A: o.k., keine Probl..."
2,beautyboysj2sgq3.onion,"[beautyboysj2sgq3.onion?p=329542, beautyboysj2...","[All the Things | Beauty of Boys, Log In ‹ The...",[Robbie in Denim Shorts | Beauty of Boys Robbi...
3,beehive6ratfqulk.onion,[beehive6ratfqulk.onion/author/jdoe/page/6],[jdoe | Beehive],[jdoe | Beehive jdoe – Page 6 – The beehive! T...
4,bfnews3u2ox4m4ty.onion,[bfnews3u2ox4m4ty.onion/article/ellencushing/b...,[14 Things BuzzFeed Tech Tried And Liked In 20...,[This story was updated Friday to reflect the ...
...,...,...,...,...
157,wikipedmkndaoiug3x3dy4p5ii4tcrawgcek7cee3mba6u...,[wikipedmkndaoiug3x3dy4p5ii4tcrawgcek7cee3mba6...,[The Hidden Wiki],[TOR Wallet - Bitcoin Wallet with integrated B...
158,wikitor7z5iiknxv7nbzo3uacz25b4lh5rxwxbrbiba5yy...,[wikitor7z5iiknxv7nbzo3uacz25b4lh5rxwxbrbiba5y...,"[The Hidden Wiki, MediaWiki API result - The H...",[TOR Wallet - Bitcoin Wallet with integrated B...
159,wikitorcwogtsifs.onion,[wikitorcwogtsifs.onion],[The Hidden Wiki],[TOR Wallet - Bitcoin Wallet with integrated B...
160,xonionsix6ncwde3.onion,[xonionsix6ncwde3.onion],[Porn Videos - XONIONS],"[Didlo, Plumber, Son, Milking, HOME, 20 minHom..."


In [13]:
df_all[df_all['domain_url']=='NA']

,domain_url,url,title,body_text


In [14]:
df_all['url_array'] = df_all['url'].apply(lambda x: np.array(x))
df_all['name_index'] = df_all['url_array'].apply(lambda x: np.argmin(x))
df_all['domain_name'] = df_all.apply(lambda x: get_domain_name(x['title'], x['name_index']), axis=1)
df_all['domain_name'] = df_all['domain_name'].apply(lambda x: custom_strip(x))
df_all.drop(columns=['url_array', 'name_index', 'title', 'url'], inplace=True)
df_all.rename(columns={'domain_name': 'title'}, inplace=True)
df_all = df_all[['domain_url', 'title', 'body_text']]
df_all

,domain_url,title,body_text
0,5figq755l7c55eopjphypkpfj5b4ap5nm6rvie2tygcwta...,dillon | PedoBum,"[dillon | PedoBum, Home, Thanks for the visit...."
1,74i677egkh3zlk6r.onion,freiheitsfoo Wiki | Category / GroupFooter | V...,"[# Erster Punkt, Powerweb A: o.k., keine Probl..."
2,beautyboysj2sgq3.onion,March | 2017 | Beauty of Boys,[Robbie in Denim Shorts | Beauty of Boys Robbi...
3,beehive6ratfqulk.onion,jdoe | Beehive,[jdoe | Beehive jdoe – Page 6 – The beehive! T...
4,bfnews3u2ox4m4ty.onion,BuzzFeed News | Breaking News | Original Repor...,[This story was updated Friday to reflect the ...
...,...,...,...
157,wikipedmkndaoiug3x3dy4p5ii4tcrawgcek7cee3mba6u...,The Hidden Wiki,[TOR Wallet - Bitcoin Wallet with integrated B...
158,wikitor7z5iiknxv7nbzo3uacz25b4lh5rxwxbrbiba5yy...,The Hidden Wiki,[TOR Wallet - Bitcoin Wallet with integrated B...
159,wikitorcwogtsifs.onion,The Hidden Wiki,[TOR Wallet - Bitcoin Wallet with integrated B...
160,xonionsix6ncwde3.onion,Porn Videos - XONIONS,"[Didlo, Plumber, Son, Milking, HOME, 20 minHom..."


In [15]:
df_all.to_csv('../data/dataset-'+date+'.csv', index=False)